In [1]:
import pandas as pd
import os
import csv
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
from scipy.ndimage.filters import median_filter

In [2]:
from Experimentos import Experimentos

[Errno 2] No such file or directory: '.\\Data\\5cc50C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\10cc60C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc60C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc59C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc63C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc62C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc62C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc24C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\10cc24C - flir.csv'


In [3]:
for i, exp in enumerate(Experimentos):
    print(i, exp, (exp.flir.dtypes if exp.flir is not None else 0))

0 5cc50C.csv 0
1 1cc54C.csv 0
2 25cc54C.csv 0
3 10cc60C.csv 0
4 25cc60C.csv 0
5 1cc59C.csv 0
6 1cc47C.csv FLIR: tiempo       int64
FLIR: area         int64
FLIR: rango 1    float64
FLIR: rango 2    float64
FLIR: rango 3    float64
FLIR: rango 4    float64
FLIR: rango 5    float64
FLIR: rango 6    float64
FLIR: rango 7    float64
FLIR: rango 8    float64
FLIR: rango 9    float64
dtype: object
7 1cc55C.csv FLIR: tiempo       int64
FLIR: area         int64
FLIR: rango 1    float64
FLIR: rango 2    float64
FLIR: rango 3    float64
FLIR: rango 4    float64
FLIR: rango 5    float64
FLIR: rango 6    float64
FLIR: rango 7    float64
FLIR: rango 8    float64
FLIR: rango 9    float64
dtype: object
8 25cc58C.csv FLIR: tiempo       int64
FLIR: area         int64
FLIR: rango 1    float64
FLIR: rango 2    float64
FLIR: rango 3    float64
FLIR: rango 4    float64
FLIR: rango 5    float64
FLIR: rango 6    float64
FLIR: rango 7    float64
FLIR: rango 8    float64
FLIR: rango 9    float64
dtype: object


In [21]:
Experimentos[14].flir.describe()

,FLIR: tiempo,FLIR: area,FLIR: rango 1,FLIR: rango 2,FLIR: rango 3
count,1579.000000,1579.000000,1579.000000,1579.000000,1579.000000
mean,789.000000,94264.607980,0.113217,0.121274,0.212874
std,455.962352,19905.099412,0.086001,0.103316,0.164292
min,0.000000,299.000000,0.015796,0.005306,0.005876
25%,394.500000,95856.000000,0.032797,0.020537,0.039449
50%,789.000000,101932.000000,0.086612,0.102057,0.192063
75%,1183.500000,103834.000000,0.188869,0.196173,0.391706
max,1578.000000,104908.000000,0.327759,0.354756,0.443221


In [5]:
for exp in Experimentos:
    if exp.flir is not None:
        try:
            exp.flir['area_m2'] = exp.flir['FLIR: area']/numeropixeles
            exp.flir['volumen_emitido'] = exp.param['caudal'][0]*exp.flir['FLIR: tiempo']
        except:
            continue
        volumen_total = kilos_manjar/rho_manjar
        exp.flir['volumen_emitido'][exp.flir['volumen_emitido'] > volumen_total] = volumen_total
        exp.flir['espesor'] = 3*exp.flir['volumen_emitido']/(2*exp.flir['area_m2'])

In [13]:
exp.get_ranges()

[24.0,
 27.377777777777776,
 30.755555555555556,
 34.133333333333333,
 37.511111111111113,
 40.888888888888886,
 44.266666666666666,
 47.644444444444446,
 51.022222222222226,
 54.399999999999999]

In [46]:
#FALTA: agregar rango de temperaturas a partir de archivo "resumen general experimentos":
#(max-min)/rangos = salto temp
#rangos=min+salto etc

In [ ]:
#FALTA: graficar tiempo en que prevalece cada rango de temperatura? y como relacionarlo a la velocidad

In [4]:
exp = Experimentos[30]

In [15]:
Experimentos[6].get_closest_points_to_intersections()

['0.9536492014866595', '0.059804062054914814', '1.0332272690781028', '-0.7738854591314837', '0.6353865292881408', '0.6837549563979712', '0.5408321558488145']
[[ inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf]]


C:\Anaconda3\envs\bootcamp\lib\site-packages\scipy\optimize\minpack.py:715: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


(0.63538652928814077,
 0.68375495639797124,
 0.5408321558488145,
 (-4.8117844353510897, -4.4228486291941369),
 (-2.5523293261054341, -2.9957322735539909))

In [14]:
Experimentos[6]

1cc47C.csv

In [7]:
exp.param.head()

,densidad manjar,modelo 3D,volumen total teorico,t amb,termocupla,caudal,angulo inclinacion,NOMBRE,Obs,Exp #,...,fecha,min,max,framei,framef,result: largo final,result: largo total,result: ancho max final,result: espesor max final,result: distancia espesor final
0,1240,hay fotos,0.00143,24.1,"si, 4 puntos","1,00E-05",10.5,10cc50C,NaN,31,...,30/09/2016 0:00,26,51,14,2020,0.368,0.442,0.305,0.015,0.25


In [6]:
Experimentos[30].flir.head()

,FLIR: tiempo,FLIR: area,FLIR: rango 1,FLIR: rango 2,FLIR: rango 3,FLIR: rango 4,FLIR: rango 5,FLIR: rango 6,FLIR: rango 7,FLIR: rango 8,FLIR: rango 9
0,0,1666,0.587635,0.063625,0.055222,0.120648259,0.049819928,0.070828331,0.027611044,0.024610,0.000000
1,1,2254,0.478261,0.053239,0.035492,0.066104703,0.022626442,0.068322981,0.20851819,0.067436,0.000000
2,2,3051,0.443461,0.043592,0.025565,0.033431662,0.018354638,0.033759423,0.106850213,0.234677,0.060308
3,3,3677,0.403590,0.039162,0.019581,0.014413924,0.010606473,0.027740005,0.071797661,0.221648,0.191460
4,4,3923,0.305379,0.037216,0.013765,0.01172572,0.011470813,0.023196533,0.071119042,0.253887,0.272241


In [8]:
def rango_temperatura(temp, exp):
    bins = exp.get_ranges()
    return np.digitize([temp], bins)[0] + 1

In [4]:
exp.rango_temperatura(34)

4

In [10]:
#GENERA PATCHES DE COLORES SEGUN TEMPERATURA
import matplotlib.patches as mpatches

pat1 = mpatches.Patch(color='C1', label='temp < 40')
pat2 = mpatches.Patch(color='C2', label='temp < 45')
pat3 = mpatches.Patch(color='C3', label='temp < 50')
pat4 = mpatches.Patch(color='C4', label='temp < 55')
pat5 = mpatches.Patch(color='C5', label='temp < 60')
pat6 = mpatches.Patch(color='C6', label='temp < 65')
pat7 = mpatches.Patch(color='C7', label='temp < 70')

numerocolores = 7
colors = ['pat{}'.format(i+1) for i in range(numerocolores)]

print(colors)

['pat1', 'pat2', 'pat3', 'pat4', 'pat5', 'pat6', 'pat7']


In [37]:
#LISTA para ESPESOR vs TIEMPO
listaporcaudal = []

for exp in Experimentos:
    if exp.flir is not None:
        try:
            holder = {} #es un diccionario 
            holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
            holder['espesor'] = exp.flir['espesor']#toda la columna
            holder['tiempo'] = exp.flir['FLIR: tiempo']#toda la columna
            holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
            holder['angulo'] = exp.param['angulo inclinacion'][0]
            holder['uso'] = exp.param['uso manjar'][0]

            listaporcaudal.append(holder)
        except:
            continue

In [27]:
#crear GRUPOS segun CAUDAL
from itertools import groupby
count = 0

#key es la lista donde estan los diccionarios de exp y DF
#group es que se agrupa segun el valor de 'caudal' en este caso

for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    count += 1
    #for exp in group:
        #muestra el nombre del grupo y los elementos que tiene
        #print(exp['nombre']) 

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [38]:
#ESPESOR vs TIEMPO agrupados por caudales
fig = plt.figure()
axes = [fig.add_subplot(3,2,i+1) for i in range(count)]
counter = 0

for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    ax = axes[counter]
    #ax.set_xlim(minimoejelargo, maximoejelargo)
    #ax.set_ylim(0, maximoejeespesor)
    ax.set_title('{}cc'.format(key*1000000))
    counter += 1
    linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
    linea2= ax.plot([],[], label='10°', color='black')
    
    for exp in group:
        #print("C{}".format(exp["rango temperatura"]))
        #if exp['uso'] == 1: 
            if exp['angulo'] == 15:
                ax.plot(exp['tiempo'], exp['espesor'], linestyle= '--')
            else:
                ax.plot(exp['tiempo'], exp['espesor'])
        
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
plt.gca().add_artist(legend1)
fig.show()

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [39]:
#LISTA para AREA vs TIEMPO
otralistaporcaudal = []

for exp in Experimentos:
    if exp.flir is not None:
        try:
            holder = {} #es un diccionario 
            holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
            holder['area'] = exp.flir['area_m2']#toda la columna
            holder['tiempo'] = exp.flir['FLIR: tiempo']#toda la columna
            holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
            holder['angulo'] = exp.param['angulo inclinacion'][0]
            holder['uso'] = exp.param['uso manjar'][0]

            otralistaporcaudal.append(holder)
        except:
            continue

In [40]:
#AREA vs TIEMPO agrupados por caudales
fig = plt.figure()
axes = [fig.add_subplot(3,2,i+1) for i in range(count)]
counter = 0

for key, group in groupby(sorted(otralistaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    ax = axes[counter]
    #ax.set_xlim(minimoejelargo, maximoejelargo)
    #ax.set_ylim(0, maximoejeespesor)
    ax.set_title('{}cc'.format(key*1000000))
    counter += 1
    
    for exp in group:
        ax.plot(exp['tiempo'], exp['area'])
        
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
plt.gca().add_artist(legend1)
fig.show()

1e-06
5e-06
1e-05
1.7e-05
2.5e-05
